In [1]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import numpy as np
from utils.plot_data import plot_sensor_measurement, plot_wind_rose, plot_co2_rose


start_date = datetime(2024, 7, 5, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 7, 6, 23, 59, 59).replace(tzinfo=timezone.utc)

filter = '10m'

sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

# processed 10min average measurement data
df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "10m_cal_corr_acropolis.parquet")).filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()


In [6]:
# raw data
df_raw = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")).filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()

In [2]:
plot_sensor_measurement(df,sensor_id,col_name="sht45_humidity",filter=filter)

In [3]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_temperature",filter=filter)

In [4]:
plot_sensor_measurement(df,sensor_id=sensor_id,col_name="gmp343_corrected",filter=filter)

In [7]:
plot_sensor_measurement(df_raw,sensor_id=sensor_id,col_name="wxt532_speed_avg",filter=filter)

In [ ]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_pressure",filter=filter)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="enclosure_bme280_temperature",filter=filter)

In [ ]:
plot_sensor_measurement(df,sensor_id=sensor_id,col_name="gmp343_raw",filter=filter)

In [ ]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_filtered",filter=filter)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id=[1,2,4,5,6,7,8,14,15],col_name="wxt532_temperature",filter=filter)

In [8]:
# KLEG
plot_co2_rose(df,df_raw, 1, 'Klenze Gymnasium')

# X
#plot_co2_rose(df,df_raw, 2, 'X')

# X
#plot_co2_rose(df,df_raw, 3, 'X')

# X
#plot_co2_rose(df,df_raw, 4, 'X')

# DLRO
plot_co2_rose(df,df_raw, 5, 'DLR')

# TUMN
plot_co2_rose(df,df_raw, 6, 'TUM')

# FB85
#plot_co2_rose(df,df_raw, 7, 'Blutenburgturm 85m')

# KRDI
plot_co2_rose(df,df_raw, 8, 'Klinikum Rechts-der-Isar')

# KNPL
plot_co2_rose(df,df_raw, 9, 'Klinikum Neuperlach')

# SWGG
plot_co2_rose(df,df_raw, 10, 'Städtisches Willi Graf Gymnasium')

# RFIN
plot_co2_rose(df,df_raw, 11, 'Rathaus Finsing')

# WKRT
plot_co2_rose(df,df_raw, 12, 'Walter-Klingenbeck-Realschule')

# RFEL
plot_co2_rose(df,df_raw, 13, 'Rathaus Feldkirchen')

# FB48
#plot_co2_rose(df,df_raw, 14, 'Blutenburgturm 48m')

# SWMZ
plot_co2_rose(df,df_raw, 15, 'Stadtwerke München Zentrale')

# SMAI
plot_co2_rose(df,df_raw, 16, 'Schule Maisach')

# KBOG
#plot_co2_rose(df,df_raw, 17, 'Klinikum Bogenhausen')

# SBBG
plot_co2_rose(df,df_raw, 18, 'Städtisches Bertolt Brecht Gymnasium')

# X
#plot_o2_rose(df,df_raw, 19, 'X')

# LGRO
plot_co2_rose(df,df_raw, 20, 'LMU Großhadern')



In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="wxt532_direction_avg",filter=filter)

In [ ]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_humidity",filter=filter)

In [ ]:
plot_sensor_measurement(df,sensor_id,col_name="sht45_temperature",filter=filter)

In [ ]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_temperature",filter=filter)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_cpu_usage",filter=filter)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_memory_usage",filter=filter)


In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_disk_usage",filter=filter)


In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="raspi_cpu_temperature",filter=filter)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="enclosure_bme280_humidity",filter=filter)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="enclosure_bme280_pressure",filter=filter)

# Calibration Data

In [ ]:
sensor_id = [4,17,19]

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bottle_id",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_filtered",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_sht45_humidity",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_sht45_temperature",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_temperature",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bme280_temperature",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bme280_humidity",filter=None)

In [ ]:
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_bme280_pressure",filter=None)

In [ ]:
# Plot 400ppm Calibration Bottle 
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_filtered",filter=None,cut_above= 450,cut_below=350)

In [ ]:
# Plot 800ppm Calibration Bottle 
plot_sensor_measurement(df_raw,sensor_id,col_name="cal_gmp343_filtered",filter=None,cut_below=460)